In [1]:
import sqlite3
import numpy as np
import pandas as pd

# Number of samples
n = 1000

# Nominal data: Sensor Types
sensor_types = ['Soil Moisture', 'Temperature', 'Humidity', 'Light Intensity', 'Crop Health']
sensor_type_data = np.random.choice(sensor_types, n)

# Ratio data: Sensor Measurements
sensor_measurements = np.random.uniform(0.0, 100.0, n)

# Create DataFrame
df = pd.DataFrame({
    'Sensor_Type': sensor_type_data,
    'Sensor_Measurement': sensor_measurements,
})

# Create SQLite database
conn = sqlite3.connect('SmartFarmMonitoringDB.db')
cursor = conn.cursor()

# Create Sensors table
cursor.execute('''
    CREATE TABLE Sensors (
        Sensor_ID INTEGER PRIMARY KEY,
        Sensor_Type TEXT,
        Sensor_Measurement REAL
    );
''')

# Insert random data into Sensors table
for i in range(n):
    cursor.execute('INSERT INTO Sensors (Sensor_Type, Sensor_Measurement) VALUES (?, ?)',
                   (df['Sensor_Type'].iloc[i], df['Sensor_Measurement'].iloc[i]))

# Commit changes and close connection
conn.commit()
conn.close()

# Example Query to check the number of rows and columns
conn = sqlite3.connect('SmartFarmMonitoringDB.db')
result = pd.read_sql_query('SELECT * FROM Sensors;', conn)
conn.close()

# Display the shape of the result (number of rows and columns)
print("Number of Rows and Columns in the 'Sensors' Table:")
print(result.shape)
print(result)


Number of Rows and Columns in the 'Sensors' Table:
(1000, 3)
     Sensor_ID      Sensor_Type  Sensor_Measurement
0            1         Humidity           86.822663
1            2  Light Intensity            6.490895
2            3      Temperature           81.948122
3            4  Light Intensity           38.848895
4            5    Soil Moisture           84.235440
..         ...              ...                 ...
995        996      Temperature           95.457670
996        997      Temperature           83.407062
997        998      Crop Health            3.661837
998        999  Light Intensity           73.495922
999       1000    Soil Moisture           33.853360

[1000 rows x 3 columns]


In [11]:
import sqlite3
import numpy as np
import pandas as pd
from faker import Faker

# Number of samples
n = 1000

# Nominal data: Sensor Types, Farms' Location
sensor_types = ['Soil Moisture', 'Temperature', 'Humidity', 'Light Intensity', 'Crop Health']
sensor_type_data = np.random.choice(sensor_types, n)

# Ordinal data: Farms' Rating
farm_ratings = ['Low', 'Medium', 'High']
farm_rating_data = np.random.choice(farm_ratings, n, p=[0.3, 0.4, 0.3])

# Ratio data: Sensor Measurements, Farms' Area (Assuming area is a ratio data)
sensor_measurements = np.random.uniform(0.0, 100.0, n)
farm_areas = np.random.uniform(10.0, 1000.0, n)

# Create DataFrame for Sensors and Farms
df_sensors = pd.DataFrame({
    'Sensor_Type': sensor_type_data,
    'Sensor_Measurement': sensor_measurements,
})

df_farms = pd.DataFrame({
    'Farm_Name': [fake.company() for _ in range(n)],
    'Location': [fake.city() for _ in range(n)],
    'Farm_Rating': farm_rating_data,
    'Farm_Area': farm_areas
})

# Create SQLite database
conn = sqlite3.connect('SmartFarmMonitoringDB.db')
cursor = conn.cursor()

# Drop the existing Sensors, Farms, and SensorFarmsJoin tables
cursor.execute('DROP TABLE IF EXISTS Sensors;')
cursor.execute('DROP TABLE IF EXISTS Farms;')
cursor.execute('DROP TABLE IF EXISTS SensorFarmsJoin;')

# Create Sensors table
cursor.execute('''
    CREATE TABLE Sensors (
        Sensor_ID INTEGER PRIMARY KEY,
        Sensor_Type TEXT,
        Sensor_Measurement REAL
    );
''')

# Insert random data into Sensors table
for i in range(n):
    cursor.execute('INSERT INTO Sensors (Sensor_Type, Sensor_Measurement) VALUES (?, ?)',
                   (df_sensors['Sensor_Type'].iloc[i], df_sensors['Sensor_Measurement'].iloc[i]))

# Create Farms table
cursor.execute('''
    CREATE TABLE Farms (
        Farm_ID INTEGER PRIMARY KEY,
        Farm_Name TEXT,
        Location TEXT,
        Farm_Rating TEXT,
        Farm_Area REAL
    );
''')

# Insert random data into Farms table
for i in range(n):
    cursor.execute('INSERT INTO Farms (Farm_Name, Location, Farm_Rating, Farm_Area) VALUES (?, ?, ?, ?)',
                   (df_farms['Farm_Name'].iloc[i], df_farms['Location'].iloc[i],
                    df_farms['Farm_Rating'].iloc[i], df_farms['Farm_Area'].iloc[i]))

# Perform a JOIN between Sensors and Farms tables
cursor.execute('''
    CREATE TABLE SensorFarmsJoin AS
    SELECT Sensors.Sensor_ID, Sensors.Sensor_Type, Sensors.Sensor_Measurement,
           Farms.Farm_Name, Farms.Location, Farms.Farm_Rating, Farms.Farm_Area
    FROM Sensors
    INNER JOIN Farms ON Sensors.Sensor_ID = Farms.Farm_ID;
''')

# Execute the query and fetch the result
result = pd.read_sql_query('SELECT * FROM SensorFarmsJoin;', conn)

# Commit changes and close connection
conn.commit()
conn.close()

# Display the result
print("Result of JOIN operation:")
print(result)


Result of JOIN operation:
     Sensor_ID      Sensor_Type  Sensor_Measurement  \
0            1         Humidity           67.354237   
1            2      Crop Health           55.574849   
2            3         Humidity           39.413064   
3            4      Crop Health           40.328362   
4            5      Temperature           40.837223   
..         ...              ...                 ...   
995        996    Soil Moisture           64.916012   
996        997         Humidity           93.729359   
997        998    Soil Moisture           33.959551   
998        999  Light Intensity           53.186514   
999       1000      Temperature           46.712875   

                        Farm_Name           Location Farm_Rating   Farm_Area  
0                 Edwards-Morales         Monicabury        High  791.034874  
1           Bailey, Barr and Hart       East Heather        High   47.726990  
2                       Kline PLC      Patriciamouth      Medium   86.155686

In [12]:
import sqlite3
import numpy as np
import pandas as pd
from faker import Faker

# Instantiate Faker for generating fake data
fake = Faker()

# Number of samples
n = 1000

# Nominal data: Sensor Types, Farms' Location
sensor_types = ['Soil Moisture', 'Temperature', 'Humidity', 'Light Intensity', 'Crop Health']
sensor_type_data = np.random.choice(sensor_types, n)

# Ordinal data: Farms' Rating
farm_ratings = ['Low', 'Medium', 'High']
farm_rating_data = np.random.choice(farm_ratings, n, p=[0.3, 0.4, 0.3])

# Ratio data: Sensor Measurements, Farms' Area (Assuming area is a ratio data)
sensor_measurements = np.random.uniform(0.0, 100.0, n)
farm_areas = np.random.uniform(10.0, 1000.0, n)

# Create DataFrame for Sensors and Farms
df_sensors = pd.DataFrame({
    'Sensor_Type': sensor_type_data,
    'Sensor_Measurement': sensor_measurements,
})

df_farms = pd.DataFrame({
    'Farm_Name': [fake.company() for _ in range(n)],
    'Location': [fake.city() for _ in range(n)],
    'Farm_Rating': farm_rating_data,
    'Farm_Area': farm_areas
})

# Create SQLite database
conn = sqlite3.connect('SmartFarmMonitoringDB.db')
cursor = conn.cursor()

# Drop the existing Sensors, Farms, and SensorFarmsJoin tables
cursor.execute('DROP TABLE IF EXISTS Sensors;')
cursor.execute('DROP TABLE IF EXISTS Farms;')
cursor.execute('DROP TABLE IF EXISTS SensorFarmsJoin;')

# Create Sensors table
cursor.execute('''
    CREATE TABLE Sensors (
        Sensor_ID INTEGER PRIMARY KEY,
        Sensor_Type TEXT,
        Sensor_Measurement REAL
    );
''')

# Insert random data into Sensors table
for i in range(n):
    cursor.execute('INSERT INTO Sensors (Sensor_Type, Sensor_Measurement) VALUES (?, ?)',
                   (df_sensors['Sensor_Type'].iloc[i], df_sensors['Sensor_Measurement'].iloc[i]))

# Create Farms table
cursor.execute('''
    CREATE TABLE Farms (
        Farm_ID INTEGER PRIMARY KEY,
        Farm_Name TEXT,
        Location TEXT,
        Farm_Rating TEXT,
        Farm_Area REAL
    );
''')

# Insert random data into Farms table
for i in range(n):
    cursor.execute('INSERT INTO Farms (Farm_Name, Location, Farm_Rating, Farm_Area) VALUES (?, ?, ?, ?)',
                   (df_farms['Farm_Name'].iloc[i], df_farms['Location'].iloc[i],
                    df_farms['Farm_Rating'].iloc[i], df_farms['Farm_Area'].iloc[i]))

# Perform a JOIN between Sensors and Farms tables
cursor.execute('''
    CREATE TABLE SensorFarmsJoin AS
    SELECT Sensors.Sensor_ID, Sensors.Sensor_Type, Sensors.Sensor_Measurement,
           Farms.Farm_Name, Farms.Location, Farms.Farm_Rating, Farms.Farm_Area
    FROM Sensors
    INNER JOIN Farms ON Sensors.Sensor_ID = Farms.Farm_ID;
''')

# Execute the query and fetch the result
result = pd.read_sql_query('SELECT * FROM SensorFarmsJoin;', conn)

# Example Query 1: Select all rows from Sensors table
query1 = 'SELECT * FROM Sensors;'
result1 = pd.read_sql_query(query1, conn)
print("\nExample Query 1:")
print(result1)

# Example Query 2: Select specific columns from Farms table
query2 = 'SELECT Farm_Name, Location FROM Farms;'
result2 = pd.read_sql_query(query2, conn)
print("\nExample Query 2:")
print(result2)

# Example Query 3: Select average Farm_Area from Farms table for each Farm_Rating
query3 = 'SELECT Farm_Rating, AVG(Farm_Area) as Avg_Area FROM Farms GROUP BY Farm_Rating;'
result3 = pd.read_sql_query(query3, conn)
print("\nExample Query 3:")
print(result3)

# Commit changes and close connection
conn.commit()
conn.close()

# Display the result of JOIN operation
print("\nResult of JOIN operation:")
print(result)



Example Query 1:
     Sensor_ID      Sensor_Type  Sensor_Measurement
0            1  Light Intensity           69.415626
1            2      Crop Health           91.049496
2            3      Temperature           28.390417
3            4  Light Intensity           32.585313
4            5  Light Intensity           19.298491
..         ...              ...                 ...
995        996      Temperature           97.253846
996        997    Soil Moisture           38.486900
997        998    Soil Moisture           68.236670
998        999    Soil Moisture           39.902837
999       1000      Crop Health            5.829886

[1000 rows x 3 columns]

Example Query 2:
                       Farm_Name         Location
0              Martinez and Sons       Garciafort
1                   Miles-Branch   Lake Tracieton
2                       King Ltd       Jamesmouth
3    Coleman, Russell and Howard      Hillborough
4               Gilmore and Sons         Allenton
..             